In [26]:
import importlib
import sys
import os
import PIL
import json

try:
    wave
except NameError:
    import wave
from tqdm.notebook import tqdm

import za_lib
importlib.reload(sys.modules["za_images"])
importlib.reload(sys.modules["cdi_images"])
importlib.reload(za_lib)
importlib.reload(sys.modules["za_scripts"])
importlib.reload(sys.modules["cdi_filesystem"])
importlib.reload(sys.modules["cdi_audio"])

from struct_stream import StructStream

game = za_lib.Game("Zeldas Adventure (Europe).dat")

parsing file:   0%|          | 0/1 [00:00<?, ?it/s]

building sectors:   0%|          | 0/220024 [00:00<?, ?it/s]

building files:   0%|          | 0/12 [00:00<?, ?it/s]

searching files for modules:   0%|          | 0/12 [00:00<?, ?it/s]

searching cdi_zelda for modules:   0%|          | 0/65 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [27]:
game._exportCurioEnemyStats("export/curiosities")

  0%|          | 0/489 [00:00<?, ?it/s]

Malformed animation table: expected 148 bytes, found 146
	Above errors happened for cell o28
hasSprites false when sprite is present. Cell: s501
hasSprites false when sprite is present. Cell: s501


In [28]:

game.export("export")
game.exportJustScripts("export/scripts")

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

  0%|          | 0/489 [00:00<?, ?it/s]

Failed to make metadata images for description 0 on cell s305: No palette data found.
Failed to make metadata images for description 0 on cell s306: No palette data found.
Failed to make metadata images for description 2 on cell s319: No palette data found.
Failed to make metadata images for description 0 on cell s707: No palette data found.


  0%|          | 0/489 [00:00<?, ?it/s]

In [6]:
from za_filesystem import ResourceTree
from typing import List

actorDataRaw = game._underFiles.subFiles["s121"].getRecord(4, kind="data")
actorDataTree = ResourceTree.parseFromStream(StructStream(actorDataRaw, endianPrefix=">"))
bossData: List[int] = [s.peek("I") for s in actorDataTree.children["kp_init"].elements]
bossWeaponData: List[int] = [s.peek("I") for s in actorDataTree.children["wp_cmds"].elements]

for i, entry in enumerate(bossData):
    y = (entry & 0xFF) * 2
    x = ((entry >> 8) & 0xFF) * 2
    command = (entry >> 16) & 0xFFFF
    if command == 1:
        command = "setActorFieldsAndAdvance"
    elif command == 2:
        command = "setActorPosition"
    elif command == 3:
        command = "savePointerToNextCommand"
    
    bossData[i] = {"x": x, "y": y, "command": command}

for i, entry in enumerate(bossWeaponData):
    x = (entry >> 8) & 0xFF
    y = entry & 0xFF
    command = (entry >> 16) & 0xFFFF

    if command == 1:
        command = "setActorFieldsAndAdvance"
    elif command == 2:
        command = "setActorMiscDataCoords"
    elif command == 3:
        command = "setVectorAsYXCoord"
    elif command == 4:
        command = "setActorField0x28ToY"
    elif command == 5:
        command = "setHealthToYAndField0x31ToXAsWeirdBool"
    elif command == 6:
        command = "setDescriptionIndexToY"

    bossWeaponData[i] = {"x": x, "y": y, "command": command}


bossData, bossWeaponData

([{'x': 256, 'y': 256, 'command': 'setActorFieldsAndAdvance'},
  {'x': 388, 'y': 306, 'command': 'setActorPosition'},
  {'x': 256, 'y': 256, 'command': 'savePointerToNextCommand'},
  {'x': 256, 'y': 256, 'command': 7},
  {'x': 256, 'y': 256, 'command': 8},
  {'x': 284, 'y': 296, 'command': 5},
  {'x': 256, 'y': 258, 'command': 7},
  {'x': 256, 'y': 258, 'command': 8},
  {'x': 256, 'y': 260, 'command': 4},
  {'x': 256, 'y': 256, 'command': 6},
  {'x': 256, 'y': 256, 'command': 9},
  {'x': 256, 'y': 264, 'command': 4},
  {'x': 256, 'y': 256, 'command': 7},
  {'x': 256, 'y': 256, 'command': 8},
  {'x': 388, 'y': 336, 'command': 5},
  {'x': 472, 'y': 296, 'command': 5},
  {'x': 256, 'y': 258, 'command': 7},
  {'x': 256, 'y': 258, 'command': 8},
  {'x': 256, 'y': 260, 'command': 4},
  {'x': 256, 'y': 256, 'command': 6},
  {'x': 256, 'y': 256, 'command': 9},
  {'x': 256, 'y': 264, 'command': 4},
  {'x': 256, 'y': 256, 'command': 7},
  {'x': 256, 'y': 256, 'command': 8},
  {'x': 388, 'y': 356

In [406]:
# Looking into parsing weapon data

invDataRaw = game._mainFile.subFiles["invent"].getRecord(1, "data")
invDataTree = ResourceTree.parseFromStream(StructStream(invDataRaw, endianPrefix=">"))
weaponNames = [s.peekNullTermString().decode('ascii') for s in invDataTree.children["labels"].elements]
display(weaponNames)

# Data Record 0 is weapon info
# Data Record 1 is unknown, but probably sprites?
# Audio Record 0 is unknown, but probably sound effects?

for i, name in enumerate(weaponNames):
    commonName = za_lib.SPELL_LOOKUP[i + 1]
    weaponDataRaw = game._mainFile.subFiles[name].getRecord(0, "data")
    weaponDataTree = ResourceTree.parseFromStream(StructStream(weaponDataRaw, endianPrefix=">"))
    #display(weaponDataTree.children)

    weaponDescriptions = [za_lib.ActorDescription(s.copy()) for s in weaponDataTree.children["sp_desc"].elements]
    weaponGroups = [za_lib.SpriteGroup(s.copy()) for s in weaponDataTree.children["sp_groups"].elements]
    
    if len(weaponDescriptions) > 1:
        print(commonName + ": Multiple weapon descriptions! Using first one.")
    #display(weaponDescriptions, weaponGroups)
    desc = weaponDescriptions[0]
    print("{}: damage={}\tbonusDamage(??)={}".format(commonName, desc.baseDamageOrPurchasePrice_maybe, desc.bonusDamage))

weaponDescriptions[0].__dict__

['wp01',
 'wp02',
 'wp03',
 'wp04',
 'wp05',
 'wp06',
 'wp07',
 'wp08',
 'wp09',
 'wp10',
 'wp11',
 'wp12',
 'wp13',
 'wp14',
 'wp04',
 'wp04',
 'wp17',
 'wp18',
 'wp19',
 'wp20',
 'wp21',
 'wp04',
 'wp23',
 'wp24']

Wand: damage=0	bonusDamage(??)=0
BowAndArrow: damage=26	bonusDamage(??)=70
Broadsword: damage=30	bonusDamage(??)=70
Hourglass: damage=0	bonusDamage(??)=0
Calm: damage=12	bonusDamage(??)=70
Feather: damage=20	bonusDamage(??)=70
RingsOfFire: damage=34	bonusDamage(??)=0
Firestorm: damage=18	bonusDamage(??)=0
GoldNecklace: damage=16	bonusDamage(??)=70
Hammer: damage=46	bonusDamage(??)=0
JadeAmulet: damage=42	bonusDamage(??)=0
Joust: damage=38	bonusDamage(??)=70
JadeRing: damage=20	bonusDamage(??)=70
Dagger: damage=26	bonusDamage(??)=70
LeatherBook: damage=0	bonusDamage(??)=0
EnergyOrb: damage=0	bonusDamage(??)=0
Noise: damage=30	bonusDamage(??)=70
OpalAmulet: damage=0	bonusDamage(??)=0
Pyros: damage=20	bonusDamage(??)=0
RoarStick: damage=41	bonusDamage(??)=70
ShortAxe: damage=40	bonusDamage(??)=0
Trident: damage=0	bonusDamage(??)=0
TurquoiseRing: damage=40	bonusDamage(??)=40
Boomerang: damage=19	bonusDamage(??)=20


{'size': Coords(x=28, y=28),
 'groupCount': 1,
 'maxHealth_maybe': 0,
 'useCostOrDefense': 1,
 'baseDamageOrPurchasePrice_maybe': 19,
 'unk_0x14': 0,
 'unk_0x15': 0,
 'collisionSamplePoints': [Coords(x=14, y=6), Coords(x=14, y=21)],
 'unk_0x24': 0,
 'bonusDamage': 20,
 'unk_0x2b': 0,
 'unk_0x2c': 0,
 'unk_0x2a': 0,
 'type_maybe': 'Hazzard',
 'weakToSpell': 'None',
 'interactsWithItem': 'GoldTrumpet',
 'groups': None,
 'scripts': None,
 'unknownFieldFromSpriteTree': None,
 'extraUnusedGroups': None,
 '_cachedHashOfGroups': None,
 'commonName': None}

In [622]:
for cell in game.cells():
    if cell._vectorData != None:
        print(cell.name, "vector")
    if cell._tableData != None:
        print(cell.name, "table")

  0%|          | 0/489 [00:00<?, ?it/s]

aa15 vector
aa15 table
ac18 vector
ac18 table
e7 vector
e7 table
f26 vector
f26 table
g11 vector
g11 table
g9 vector
g9 table
gl1 vector
gl1 table
h29 vector
h29 table
h9 vector
h9 table
j11 vector
j11 table
j15 vector
j15 table
j17 vector
j17 table
j22 vector
j22 table
j22a vector
j22a table
j24 vector
j24 table
k13 vector
k13 table
k13a vector
k13a table
k13b vector
k13b table
k14 vector
k14 table


KeyboardInterrupt: 

In [645]:
g9 = game.getCell("i23")
g9.showActors()
g9.actors[0].animation.__dict__


Actor 0


{'descIndex': 0,
 'spawnCoords': (134, 102),
 'health': 60,
 'unk_0x24': 0,
 'direction': 'UP',
 'animationType': 'UnknownType2',
 'unk_0x28': 0,
 'unk_0x30': 0,
 'unk_0x31': 0,
 'unk_0x32': 0,
 'unk_0x34': 0,
 'unk_0x35': 0,
 'description': <za_lib.ActorDescription at 0x28528ec4ad0>,
 'animation': None}

Actor 1


{'descIndex': 0,
 'spawnCoords': (70, 144),
 'health': 60,
 'unk_0x24': 0,
 'direction': 'UP',
 'animationType': 'UnknownType2',
 'unk_0x28': 0,
 'unk_0x30': 0,
 'unk_0x31': 0,
 'unk_0x32': 0,
 'unk_0x34': 0,
 'unk_0x35': 0,
 'description': <za_lib.ActorDescription at 0x28528ec4ad0>,
 'animation': None}

Actor 2


...

Description 0


{'size': (60, 53),
 'groupCount': 4,
 'maxHealth_maybe': 60,
 'useCostOrDefense': 10,
 'baseDamageOrPurchasePrice_maybe': 45,
 'unk_0x14': 0,
 'unk_0x15': 0,
 'collisionSamplePoints': [(24, 37), (30, 37)],
 'unk_0x24': 4,
 'bonusDamage': 5,
 'unk_0x2b': 0,
 'unk_0x2c': 0,
 'unk_0x2a': 1,
 'type_maybe': 'EnemyOrSwitch',
 'weakToSpell': 'None',
 'interactsWithItem': 'NoneOrVialOfAcid',
 'groups': [<za_lib.SpriteGroup at 0x28528ec4a90>,
 'scripts': <za_scripts.ScriptSet at 0x28528ec4450>,
 'unknownFieldFromSpriteTree': 5491720,
 'extraUnusedGroups': None,
 '_cachedHashOfGroups': None,
 'commonName': 'enemy.goriya'}

Description 1


{'size': (32, 32),
 'groupCount': 1,
 'maxHealth_maybe': 0,
 'useCostOrDefense': 0,
 'baseDamageOrPurchasePrice_maybe': 40,
 'unk_0x14': 0,
 'unk_0x15': 0,
 'collisionSamplePoints': [(14, 15), (5, 17)],
 'unk_0x24': 0,
 'bonusDamage': 0,
 'unk_0x2b': 0,
 'unk_0x2c': 0,
 'unk_0x2a': 48,
 'type_maybe': 'Normal',
 'weakToSpell': 'None',
 'interactsWithItem': 'NoneOrVialOfAcid',
 'groups': [<za_lib.SpriteGroup at 0x284a9f7be10>],
 'scripts': <za_scripts.ScriptSet at 0x28528ec5590>,
 'unknownFieldFromSpriteTree': 5516260,
 'extraUnusedGroups': None,
 '_cachedHashOfGroups': None,
 'commonName': 'enemy.goriya.projectile'}

AttributeError: 'NoneType' object has no attribute '__dict__'

In [787]:
import math

@dataclass
class BitCounter:
    counts: List[int]
    total: int = 0

    def update(self, value: int):
        self.total += 1
        for i in range(len(self.counts)):
            if (1 << i) & value != 0:
                self.counts[i] += 1

    def ratios(self) -> List[float]:
        return [c / self.total for c in self.counts]
    
    def printStats(self):
        averages = self.ratios()
        print("null count hypothesis:", self.total / 2, "+/-", math.sqrt(self.total*0.5*0.5))
        for i in range(len(averages)):
            ones = pow(1 - averages[i], 2) * self.counts[i]
            zeroes = pow(averages[i], 2) * (self.total - self.counts[i])
            stdev = math.sqrt(ones + zeroes)
            print(i, "mean:", averages[i], "count:", self.counts[i], "+/-", stdev)

uniqueDescValues = set()
uniqueGroupValues = set()
descCounter = BitCounter([0] * 32)
groupCounter = BitCounter([0] * 32)
for cell in game.cells(duplicates=True):
    for desc in cell.descriptions:
        descCounter.update(desc.unknownFieldFromSpriteTree)
        uniqueDescValues.add(desc.unknownFieldFromSpriteTree)
        for group in desc.groups:
            groupCounter.update(group.unknownFieldFromSpriteTree)
            uniqueGroupValues.add(group.unknownFieldFromSpriteTree)

with open("spriteFieldValues_desc.csv", "w") as f:
    f.write("values\n")
    for value in uniqueDescValues:
        f.write("{}\n".format(value))
with open("spriteFieldValues_group.csv", "w") as f:
    f.write("values\n")
    for value in uniqueGroupValues:
        f.write("{}\n".format(value))

descCounter.printStats()
groupCounter.printStats()

  0%|          | 0/489 [00:00<?, ?it/s]

null count hypothesis: 372.0 +/- 13.638181696985855
0 mean: 0.0 count: 0 +/- 0.0
1 mean: 0.0 count: 0 +/- 0.0
2 mean: 0.21505376344086022 count: 160 +/- 11.206756794428188
3 mean: 0.7432795698924731 count: 553 +/- 11.914965289477866
4 mean: 0.39381720430107525 count: 293 +/- 13.32709867674825
5 mean: 0.553763440860215 count: 412 +/- 13.559109940021559
6 mean: 0.478494623655914 count: 356 +/- 13.625561051879465
7 mean: 0.489247311827957 count: 364 +/- 13.635027630871294
8 mean: 0.44623655913978494 count: 332 +/- 13.559109940021557
9 mean: 0.4368279569892473 count: 325 +/- 13.528891823741317
10 mean: 0.28360215053763443 count: 211 +/- 12.294712124997442
11 mean: 0.5524193548387096 count: 411 +/- 13.563024926663312
12 mean: 0.3602150537634409 count: 268 +/- 13.094363886474127
13 mean: 0.4529569892473118 count: 337 +/- 13.577683698763053
14 mean: 0.7083333333333334 count: 527 +/- 12.397916491626056
15 mean: 0.6545698924731183 count: 487 +/- 12.970137330251804
16 mean: 0.6008064516129032 co

In [758]:
def findDescWithBit(bit: int, maxFind = 100000) -> List[za_lib.ActorDescLocation]:
    mask = 1 << bit
    ret = []
    for cell in game.cells(duplicates=True):
        for i, desc in enumerate(cell.descriptions):
            if mask & desc.unknownFieldFromSpriteTree != 0:
                ret.append((
                    za_lib.ActorDescLocation(cell.info.isOverworld, cell.name, i),
                    desc.commonName
                ))
                if len(ret) >= maxFind:
                    return ret
    return ret

itemsWithBit2 = [name for _, name in findDescWithBit(2) if "item." in name]
itemsWithoutBit2 = []
for name in game.spriteNames:
    if "item." in name and name not in itemsWithBit2:
        itemsWithoutBit2.append(name)
itemsWithoutBit2, itemsWithBit2

  0%|          | 0/489 [00:00<?, ?it/s]

(['item.compass.s1',
  'item.compass.s2',
  'item.compass.s6',
  'item.map.s1',
  'item.map.s7',
  'item.treasure.alligatorBoots',
  'item.treasure.bone',
  'item.treasure.deepFriedOctorokLegs',
  'item.treasure.diamond',
  'item.treasure.feather',
  'item.treasure.fishingNet',
  'item.treasure.fullWaterBottle',
  'item.treasure.goldKey',
  'item.treasure.goldenBoots',
  'item.treasure.goldenKey',
  'item.treasure.ladder',
  'item.treasure.magicShield',
  'item.treasure.plank',
  'item.treasure.redBoots',
  'item.treasure.redBow',
  'item.treasure.saltcellar',
  'item.treasure.vialOfWinds',
  'item.weapon.pyros',
  'item.weapon.bowAndArrow',
  'item.weapon.firestorm',
  'item.weapon.goldNecklace',
  'item.weapon.hammer',
  'item.weapon.hourglass',
  'item.weapon.jadeRing',
  'item.weapon.joust',
  'item.weapon.leatherBook',
  'item.weapon.ringsOfFire',
  'item.weapon.roarStick',
  'item.weapon.trident',
  'item.weapon.turquoiseRing',
  'item.weapon.wand',
  'item.treasure.silverTrumpet

In [788]:
for cell in game.cells(duplicates=True):
    for desc in cell.descriptions:
        if desc.unknownFieldFromSpriteTree < 0x53B000:
            print(cell.name)

  0%|          | 0/489 [00:00<?, ?it/s]

f21
f25
f27
f28
g25
g26
g27
g28
g29
g30
g31
g32
g32
h20
h21
h25
h26
h27
h28
i21
i22
i25
i26
j10
j16
j20
j21
j25
j26
k16
k17
k18
k19
l10
l16
l18
l18
m10
m10
m12
m14
m15
m16
m17
m18
n13
n14
n16
n8
o12
o12
o15
o17
p11
p13
p13
q10
q13
q8
q9
r15
r15
r5
r6
r7
r8
s7
s7
s8
t14
t14
t8
u15
u15
v10
v12
v12
v8
v9
w12
w16
x5
x6
x7
x9
y3
y4
z2
g32
g32
s103
s104
s104
s105
s105
s106
s110
s111
s115
s115
s116
s116
s116
s119


In [800]:
import json
tumblebotSubVariants = {}
for desc in game.getActorsByName("enemy.tumbleBot", 0):
    field = desc.unknownFieldFromSpriteTree
    if field not in tumblebotSubVariants:
        tumblebotSubVariants[field] = []
    tumblebotSubVariants[field].append(desc)
display(tumblebotSubVariants)

subVariantJson = {}
for name, variants in tumblebotSubVariants.items():
    subVariantJson[name] = [
        json.dumps(t, default=za_lib._cellSerializer, indent=2)
        for t in variants
    ]

print(subVariantJson[5390632][0] == subVariantJson[5390632][1])
print(subVariantJson[5390632][0] == subVariantJson[5452200][0])
for name in subVariantJson:
    with open("subvariant{}.json".format(name), "w") as f:
        f.write(subVariantJson[name][0])

{5390632: [<za_lib.ActorDescription at 0x28550bcadd0>,
 5452200: [<za_lib.ActorDescription at 0x28550b7aa50>,
 5558408: [<za_lib.ActorDescription at 0x2854fb6d390>],
 5451848: [<za_lib.ActorDescription at 0x284936d4e90>,
  <za_lib.ActorDescription at 0x28486222250>]}

True
False


In [11]:
for cell in game.cells(duplicates=True):
    for desc in cell.descriptions:
        if desc.lootDropped not in ["Nothing", "Random"]:
            print(cell.name, desc.commonName, desc.lootDropped)

  0%|          | 0/489 [00:00<?, ?it/s]

gl4 enemy.dragonfly Heart
gl4 enemy.dragonfly Heart


In [786]:
import za_images
spriteData = game._overFiles.subFiles["d24"].getRecord(5, "data")
spriteDataStream = StructStream(spriteData, endianPrefix=">")
pointers, rootUnkValue = za_images.unpackPointerArray(spriteDataStream)

desc0pointers, desc0UnkValue = za_images.unpackPointerArray(pointers[0])
print(desc0UnkValue & 0xFFFC)
print(len(pointers[0]), len(pointers[1]))
print(spriteDataStream._cursor, len(spriteDataStream._data), spriteDataStream)
print(hex(desc0UnkValue & 0xFFFF))


52168
6120 1504
26064 26624 b'\x00\x00\x00\x01\x00T1\x84\x00\x00\x00\x0c\x00\x00\x00\x01\x00T1\x90\x00\x00\x00\x0c\x00\x00\x01\x90\x01\x90\x00\x01\xa0\xbc\xb9\xbb\x01|\x00\x01\xb7\xb9\xb5\xbd\x01x\x00\x02\xa0\xa0\xa0\xa2\xa3\xb4\xa5\xa0\x01x\x00\x02\xa0\xa0\xbb\xae\xa7\xa7\xba\xa0\x01x\x00\x02\xa0\xbb\xa6\xa4\xaf\xbb\xa0\xa0\x01x\x00\x02\xbd\xa3\xa4\xaf\xaa\xa0\xa0\xa0\x01t\x00\x03\xa0\xa0\xa0\xba\xa3\xb6\xaf\xb4\xbd\xa0\xa0\xa0\x01t\x00\x02\xa0\xa0\xa0\xa3\xb1\xa4\xaf\xac\x01x\x00\x02\xa0\xa0\xbc\xa6\xa9\xa4\xb4\xbd\x01x\x00\x02\xa0\xa2\xa3\xb2\xb3\xa7\xa2\xa0\x01x\x00\x02\xbd\xa3\xb8\xa4\xa7\xaa\xa0\xa0\x01x\x00\x02\xb8\xa3\xb6\xaf\xa7\xbb\xa0\xa0\x01t\x00\x03\xa0\xa0\xa0\xb7\xb9\xb0\xaf\xa4\xaa\xa0\xa0\xa0\x01t\x00\x03\xa0\xa0\xbd\xb9\xbc\xb3\xa4\xa9\xa2\xa0\xa0\xa0\x01t\x00\x03\xa0\xa0\xbd\xb7\xbe\xb0\xa8\xb1\xb7\xa0\xa0\xa0\x01t\x00\x03\xa0\xa0\xa0\xb5\xb8\xb3\xa8\xb9\xa3\xa2\xa0\xa0\x01t\x00\x03\xa0\xa0\xa0\xbd\xbc\xb0\xa7\xa6\xa3\xa3\xa2\xa0\x01x\x00\x02\xbb\xaa\xaa\xb0\xa3\xa3\

In [29]:
game.zeldaActor.description.groups[0].__dict__

{'animationFrameOrder': [0, 1, 2, 3, 4],
 'unk_0x12': 0,
 'frameDelay': 2,
 'treeHeightSamples': [(25, 56), (41, 57), (60, 54)],
 'damageSamplePoints_maybe': [],
 'sprites': [<PIL.Image.Image image mode=P size=55x57>,
  <PIL.Image.Image image mode=P size=59x55>,
  <PIL.Image.Image image mode=P size=59x51>,
  <PIL.Image.Image image mode=P size=59x53>,
  <PIL.Image.Image image mode=P size=59x53>],
 'unusedSprites': [],
 'unusedSpritePointer': 5522696,
 '_cachedHash': None}

In [42]:
valuesPerEnity = {}
for cell in game.cells(duplicates=True):
    for desc in cell.descriptions:
        if desc.commonName not in valuesPerEnity:
            valuesPerEnity[desc.commonName] = {}
        if desc.unk_0x2b not in valuesPerEnity[desc.commonName]:
            valuesPerEnity[desc.commonName][desc.unk_0x2b] = []
        if cell.name not in valuesPerEnity[desc.commonName][desc.unk_0x2b]:
            valuesPerEnity[desc.commonName][desc.unk_0x2b].append(cell.name)
        
for name in sorted(valuesPerEnity):
    values = valuesPerEnity[name]
    if len(values) == 1:
        print(name, "always has value", list(values.keys())[0], list(values.values())[0])
    else:
        print(name, "has different values on different cells:")
        for value, cellNames in values.items():
            print("\t{}".format(value), "on cells", ", ".join(cellNames))

  0%|          | 0/489 [00:00<?, ?it/s]

enemy.agwanda has different values on different cells:
	3 on cells gl5, s525
	0 on cells s516
enemy.agwanda.projectile has different values on different cells:
	0 on cells gl5
	60 on cells s525
enemy.alligatorMan always has value 0 ['r20', 's16', 't16', 'u14', 'v15', 'v18', 'w15']
enemy.alligatorMan.projectile always has value 0 ['s16', 't16']
enemy.archer always has value 0 ['s722']
enemy.archer.projectile always has value 0 ['s722']
enemy.aviana always has value 3 ['gl3', 's321']
enemy.aviana.projectile always has value 0 ['gl3', 's321']
enemy.axeMan always has value 0 ['x14', 'x15', 'x16', 'y16', 'z15']
enemy.bagoBago always has value 0 ['c8', 'o14', 'p13', 'p14', 'q14']
enemy.bagoBago.projectile always has value 0 ['c8', 'o14', 'q14']
enemy.blob always has value 0 ['r17', 't14', 't15', 'u15']
enemy.bolla always has value 0 ['s720']
enemy.bolla.projectile always has value 0 ['s720']
enemy.cactus always has value 3 ['v7', 'w9', 'x8']
enemy.cactus.projectile always has value 0 ['v7', 